# Part 2: Clients and Datasite Access

### **Reconnecting to the Datasite** -> Owen will have to configure the access credentials and policies, in order to enable Rachel to operate on the Datasite as Data Scientist

We explicitly define reset = False (default value) to **ensure persistency is restored**. In other words, when we reconnect to the Datasite, we will expect to find the “Breast Cancer Dataset” already uploaded on the Datasite. Also, we use the same value for the name parameter, namely name="cancer-research-centre". 

## **Important:**

- using the **same name** for the local development server will guarantee **continuity** and **persistency** throughout multiple instances;
- **reset=True** parameter meant to be used only the **first time**, or when we want to re-initialise the whole server

In [1]:
import syft as sy

data_site = sy.orchestra.launch(name="cancer-research-centre", reset=False)

# logging in as root client with default credentials
client = data_site.login(email="info@openmined.org", password="changethis")

#check if “Breast Cancer Dataset” is present and accessible through the available datasets

client.datasets

#It is!


CRITICAL:syft.server.server:Hash of the signing key 'be0c9...'


Using SQLiteDBConfig and sqlite:///C:\Users\duart\AppData\Local\Temp\syft\151a31aa3965414dbbf9f27d4e4108aa\db\151a31aa3965414dbbf9f27d4e4108aa_json.db


SyftInfo: You have launched a development server at http://0.0.0.0:None. It is intended only for local use.

 ### **Updating default admin credentials** 
 
Owen was initially using the credentials provided by default in PySyft. In order to setup the cancer-research-centre DataSite, he has to customise the access

In [ ]:
OWEN_EMAIL = "owen@cancer-research.science"
OWEN_PASSWD = "cancer_research_syft_admin"

#Procedure to ensure that running more than once this cell with the creation procedure doesn't result in error

def user_exists(users, email):
    """
    Check if a user with the specified email exists.

    Parameters:
        users (API module): All existing users.
        email (str): The email of the user to search for.

    Returns:
        logical (bool): True if a user with the specified email exists, False otherwise.
    """
    logical = False
    for user in users:
        if user.email == email:
            logical = True # User with the email found
            break
            
    return logical 


#All users
users = client.users
        
if user_exists(users, OWEN_EMAIL):
    print(f"User {OWEN_EMAIL} already exists!")

else:

    #setting email
    client.account.set_email(OWEN_EMAIL)
    
    # setting password. To bypass the confirmation, use the confirm=False parameter -> NOT SAFE (meant for tutorial purposes)
    client.account.set_password(OWEN_PASSWD, confirm=False)
    
    #After confirming the setting was successful, we update Owen's credentials:
         
    client.account.update(name="Owen, the Data Owner", 
                     institution="Cancer Research Centre")


In [ ]:
#test our new credentials by instantiating a new (root) client, and accessing registered users info

client = data_site.login(email=OWEN_EMAIL, password=OWEN_PASSWD)

client.users

#Worked! From now on, the next time Owen’s will connect again to the Datasite, he will use these new set of credentials.

### **Register Rachel’s Account** -> Owen needs to solve is to allow Rachel to connect to the Datasite! 

In other words,  Owen needs to add a new user on the Datasite, to be registered with the role of Data Scientist!

We can use the `client.users.create()` function, which expects the following parameters:

  -  `name` (type: str): mandatory

  -  `email` (type: str): mandatory

  -  `password` (type: str): mandatory

  -  `password_verify` (type_str): mandatory

  -  `institution` (type: str): optional

  -  `website` (type: str): optional


In [ ]:
RACHEL_EMAIL = "rachel@datascience.inst"
users = client.users

if user_exists(users, RACHEL_EMAIL):
    print(f"User {RACHEL_EMAIL} already exists!")
else:
    rachel_account_info = client.users.create(
        email="rachel@datascience.inst",
        name="Dr. Rachel Science",
        password="syftrocks",
        password_verify="syftrocks",
        institution="Data Science Institute",
        website="https://datascience_institute.research.data"
    )


In [ ]:
print(f"New User: {rachel_account_info.name} ({rachel_account_info.email}) registered as {rachel_account_info.role}")

In [ ]:
#To verify Rachel was added!
client.users

This clear distinction between the main components of an asset has the following advantages:

  1.  mock data is open-access and imposes no risks to the data owner for sharing publicly non-public information;

  2.  it creates a staging environment for the data scientist to simulate their intended study in a realistic way;

  3.  reduces liability for the data scientist, who is not responsible anymore for storing safely non-public data;

  4. enables the data owner to control how non-public assets can be used by data scientists for their study.
